In [10]:
# Importar as bibliotecas necessárias
import torch
import os
import numpy as np
import json
import pandas as pd
from pathlib import Path
from huggingface_hub import snapshot_download, notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [11]:
# Definir a chave da API do Hugging Face
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY_BRANDO")

# Realizar o login no Hugging Face (se necessário)
# notebook_login()

# Definir o caminho para salvar o modelo
mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

# Nome do modelo a ser baixado
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# Baixar o modelo (descomente se necessário)
# snapshot_download(repo_id=model_name,
#                   allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"],
#                   local_dir=mistral_models_path)

In [12]:
# Carregar os dados dos arquivos CSV
df_pacientes_reg = pd.read_csv('../data/entrada_modelo/df_pacientes_reg.csv', sep=';', encoding='utf-8')
df_diag_status = pd.read_csv('../data/entrada_modelo/df_diag_status.csv', sep=';', encoding='utf-8')
df_cond_add = pd.read_csv('../data/entrada_modelo/df_cond_add.csv', sep=';', encoding='utf-8')
df_cond_familia = pd.read_csv('../data/entrada_modelo/df_cond_familia.csv', sep=';', encoding='utf-8')
df_cons_reg = pd.read_csv('../data/entrada_modelo/df_cons_reg.csv', sep=';', encoding='utf-8')
df_surtos_reg = pd.read_csv('../data/entrada_modelo/df_surtos_reg.csv', sep=';', encoding='utf-8')
df_lab_basic = pd.read_csv('../data/entrada_modelo/df_lab_basic.csv', sep=';', encoding='utf-8')
df_lab_hemat = pd.read_csv('../data/entrada_modelo/df_lab_hemat.csv', sep=';', encoding='utf-8')
df_lab_quimica = pd.read_csv('../data/entrada_modelo/df_lab_quimica.csv', sep=';', encoding='utf-8')
df_lab_sorol = pd.read_csv('../data/entrada_modelo/df_lab_sorol.csv', sep=';', encoding='utf-8')
df_lab_aac = pd.read_csv('../data/entrada_modelo/df_lab_aac.csv', sep=';', encoding='utf-8')
df_mri = pd.read_csv('../data/entrada_modelo/df_mri.csv', sep=';', encoding='utf-8')
df_liquor = pd.read_csv('../data/entrada_modelo/df_liquor.csv', sep=';', encoding='utf-8')
df_pot_evoc = pd.read_csv('../data/entrada_modelo/df_pot_evoc.csv', sep=';', encoding='utf-8')
df_trat_dmd = pd.read_csv('../data/entrada_modelo/df_trat_dmd.csv', sep=';', encoding='utf-8')
df_trat_odr = pd.read_csv('../data/entrada_modelo/df_trat_odr.csv', sep=';', encoding='utf-8')
df_trat_out = pd.read_csv('../data/entrada_modelo/df_trat_out.csv', sep=';', encoding='utf-8')
df_vacinas_reg = pd.read_csv('../data/entrada_modelo/df_vacinas_reg.csv', sep=';', encoding='utf-8')
df_gravidez = pd.read_csv('../data/entrada_modelo/df_gravidez.csv', sep=';', encoding='utf-8')

# Lista com todos os DataFrames
df_list = [df_pacientes_reg, df_diag_status, df_cond_add, df_cond_familia, df_cons_reg, df_surtos_reg, 
           df_lab_basic, df_lab_hemat, df_lab_quimica, df_lab_sorol, df_lab_aac, df_mri, df_liquor, 
           df_pot_evoc, df_trat_dmd, df_trat_odr, df_trat_out, df_vacinas_reg, df_gravidez]

# Carregar o DataFrame de resultados
df_resultados = pd.read_csv('../data/entrada_modelo/df_resultados.csv', sep=';', encoding='utf-8')

# Função para converter string para dicionário, lidando com dicionários aninhados
def str_to_dict(details_str):
    try:
        return json.loads(details_str.replace("'", '"'))
    except json.JSONDecodeError:
        return details_str

# Aplicar a função à coluna 'resultado'
df_resultados['resultado'] = df_resultados['resultado'].apply(str_to_dict)

In [13]:
# Configurar o pipeline do modelo
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [14]:
def check_output(output):
    # TODO: Implementar a função de checagem de saída
    return output

# Função para extrair informações usando ReAct
def extrair_informacao_react(data_list, result_list):
    n_npac = 0
    for _, value in result_list.iloc[0:2].iterrows():  # Para cada NPAC
        dicionario_dados = value['resultado'] # Dicionário com os dados que serão preenchidos
        for i, table in enumerate(dicionario_dados.keys()):  # Para cada tabela do dicionario
            temp = data_list[i].reset_index(drop=True) # Seleciona a tabela de dados
            temp = temp[temp['npac'] == value['npac']].reset_index(drop=True) # Seleciona os dados do paciente

            if table == 'list_pacientes_reg' or temp.shape[0] ==0:  # Se a tabela for o registro de pacientes, pule
                continue
            else:
                item_size = len(dicionario_dados[table])
                control = item_size
                for item in dicionario_dados[table]:  # Para cada item na tabela
                    if isinstance(item, dict):  # Se a tabela tiver mais ocorrências
                        temp_chunk = temp[temp.columns[-1]].iloc[item_size-control]
                        for key in item.keys():  # Para cada item na ocorrência
                            if key == 'npac' or key.startswith('id') or key.startswith('coment'):
                                continue
                            else:
                                prompt = f'busque o dado que preenche {key} que deve ser {item[key]} com o formato descrito ao fim entre []. Este dado deve ser achado em: {temp_chunk}'
                                output = None# generator(prompt, max_length=50)[0]['generated_text']
                                output = check_output(output)
                                # Atualiza o resultado com a saída do modelo
                                result_list.loc[result_list['npac'] == value['npac'], 'resultado'][n_npac][table][item_size-control][key] = output
                        control -= 1
                    else:
                        if item == 'npac' or item.startswith('id') or item.startswith('coment'):
                            continue
                        else:
                            prompt = f'busque o dado que preenche {item} que deve ser {dicionario_dados[table][item]} com o formato descrito ao fim entre []. Este dado deve ser achado em: {temp[temp.columns[-1]].values[0]}'
                            output = None# generator(prompt, max_length=50)[0]['generated_text']
                            output = check_output(output)
                            print(prompt)
                            # Atualiza o resultado com a saída do modelo
                            result_list.loc[result_list['npac'] == value['npac'], 'resultado'][0][table][item] = output
        n_npac += 1
    return result_list

# Exemplo de uso
temp = extrair_informacao_react(df_list, df_resultados)

busque o dado que preenche dt_ris que deve ser Data de ocorrência de RIS/SRI (Síndrome Radiológica Isolada) [date] com o formato descrito ao fim entre []. Este dado deve ser achado em: evolução para forma sem particularidades (normal) ou declínio cognitivo por outra causarnrnsem clara evidência de surto neurite óptica último ano porém com queixa de piora progressiva
busque o dado que preenche dt_ris_desconhecida que deve ser Data desconhecida [check] 1. sim com o formato descrito ao fim entre []. Este dado deve ser achado em: evolução para forma sem particularidades (normal) ou declínio cognitivo por outra causarnrnsem clara evidência de surto neurite óptica último ano porém com queixa de piora progressiva
busque o dado que preenche dt_ini_prog que deve ser Data de início de progressão [date] com o formato descrito ao fim entre []. Este dado deve ser achado em: evolução para forma sem particularidades (normal) ou declínio cognitivo por outra causarnrnsem clara evidência de surto neurit